In [78]:
 # Dependencies and Setup
import pandas as pd
import numpy as np

In [79]:
# Raw data file
file_to_load = "Resources/purchase_data.csv"

In [80]:
# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [81]:
#Find the amount of unique screennames
players = len(purchase_data["SN"].value_counts())

#Find the amount of unique items
items = len(purchase_data["Item ID"].value_counts())

#Find average price for items and round to 2 points 
#after decimal
avg_price = "$" + str(round(purchase_data["Price"].mean(),2))

#Find total number of purchases
total = purchase_data["Item Name"].count()

#Sum up the cost of all of the purchased items
total_cost = "$" + str(purchase_data["Price"].sum())


In [82]:
len(purchase_data["SN"].unique())

576

In [83]:
output = {
    "Number of Unique Items": [items],
    "Average Price": [avg_price],
    "Number of Purchases": [total],
    "Total Revenue":[total_cost]
}

item_output_pd = pd.DataFrame(output, columns = ["Number of Unique Items","Average Price","Number of Purchases","Total Revenue"])

item_output_pd

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [84]:
#Find gender breakdown
purchase_data["Gender"].value_counts()

Male                     652
Female                   113
Other / Non-Disclosed     15
Name: Gender, dtype: int64

In [85]:
#Reformat and output gender data  
gender = purchase_data.drop_duplicates("SN","first")
gender_ = pd.DataFrame({"Total Count": gender["Gender"].value_counts()})
gender_["Percentage of Players"] = round((gender_["Total Count"]/ gender_["Total Count"].sum() * 100),2)
gender_reformat = gender_[["Percentage of Players","Total Count"]]

gender_reformat

,Percentage of Players,Total Count
Male,84.03,484
Female,14.06,81
Other / Non-Disclosed,1.91,11


In [86]:
#Purchase Analysis = p_a
purchase_data["Purchase Count"] = 1
purchase_analysis = purchase_data[["Gender", "Purchase Count", "Price"]]
p_a_grouped = purchase_analysis.groupby(["Gender"])

#Purchase analysis grouped = p_a_g
p_a_g_sum = p_a_grouped.sum()


p_a_g_sum["Average Purchase Price"] = round(p_a_g_sum["Price"] / p_a_g_sum["Purchase Count"],2)
p_a_g_sum["Average Purchase Price"] = p_a_g_sum["Average Purchase Price"].map("${:.2f}".format)
p_a_g_sum["Total Purchase Value"] = p_a_g_sum["Price"].map("${:.2f}".format)
p_a_g_sum["Avg Purchase Total per Person"] = p_a_g_sum["Average Purchase Price"]

p_a_final = p_a_g_sum[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Purchase Total per Person"]]

p_a_final

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Purchase Total per Person
Gender,,,,
Female,113,$3.20,$361.94,$3.20
Male,652,$3.02,$1967.64,$3.02
Other / Non-Disclosed,15,$3.35,$50.19,$3.35


In [87]:
# Establish bins for ages
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

 # Slice the data and place it into bins
purchase_data["Age Buckets"] = pd.cut(purchase_data["Age"],age_bins,labels = group_names)
purchase_data["Total Count"] = 1

age = purchase_data[["Age Buckets", "Total Count"]]
age = age.groupby(["Age Buckets"]).count()
age["Percentage of Players"] = round(age["Total Count"] / age["Total Count"].sum() * 100,2)

age

,Total Count,Percentage of Players
Age Buckets,,
<10,23,2.95
10-14,28,3.59
15-19,136,17.44
20-24,365,46.79
25-29,101,12.95
30-34,73,9.36
35-39,41,5.26
40+,13,1.67


In [88]:
#Purchasing Analysis - Age
age_data = purchase_data[["Age Buckets", "Total Count", "Price"]]
age_data_group = age_data.groupby(["Age Buckets"]).sum()
age_data_group["Purchase Count"] = age_data_group["Total Count"]
age_data_group["Average Purchase Price"] = age_data_group["Price"] / age_data_group["Total Count"]
age_data_group["Average Purchase Price"] = age_data_group["Average Purchase Price"].map("${:.2f}".format)
age_data_group["Total Purchase Value"] = age_data_group["Price"].map("${:.2f}".format)
age_data_group["Average Purchase Total per Person"] = age_data_group["Average Purchase Price"]

adg_final = age_data_group[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Average Purchase Total per Person"]]

adg_final

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Age Buckets,,,,
<10,23,$3.35,$77.13,$3.35
10-14,28,$2.96,$82.78,$2.96
15-19,136,$3.04,$412.89,$3.04
20-24,365,$3.05,$1114.06,$3.05
25-29,101,$2.90,$293.00,$2.90
30-34,73,$2.93,$214.00,$2.93
35-39,41,$3.60,$147.67,$3.60
40+,13,$2.94,$38.24,$2.94


In [89]:
#Top Spenders
top_spenders = purchase_data[["SN", "Total Count", "Price"]]
top_spenders_group = top_spenders.groupby(["SN"]).sum()
top_spenders_group_sort = top_spenders_group.sort_values("Price", ascending = False)
top_spenders_group_sort["Purchase Count"] = top_spenders_group_sort["Total Count"]
top_spenders_group_sort["Average Purchase Price"] = top_spenders_group_sort["Price"] / top_spenders_group_sort["Total Count"]
top_spenders_group_sort["Average Purchase Price"] = top_spenders_group_sort["Average Purchase Price"].map("${:.2f}".format)
top_spenders_group_sort["Total Purchase Value"] = top_spenders_group_sort["Price"].map("${:.2f}".format)

top_spenders_final = top_spenders_group_sort[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

top_spenders_final.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [90]:
#List most popular items
items = purchase_data[["Item ID","Item Name","Price","Purchase Count"]]
items_group = items.groupby(["Item ID","Item Name"]).sum()
items_group["Total Purchase Value"] = items_group["Price"]
items_group["Item Price"] = round(items_group["Price"] / items_group["Purchase Count"],2)
items_group["Total Purchase Value"] = items_group["Total Purchase Value"].map("${:.2f}".format)

items_final = items_group[["Purchase Count", "Item Price", "Total Purchase Value"]]
items_real_final = items_final.sort_values("Purchase Count", ascending = False)
items_real_final.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,$50.76
145,Fiery Glass Crusader,9,4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,$31.77
82,Nirvana,9,4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,$8.16


In [91]:
#Find the most profitable item
items = purchase_data[["Item ID","Item Name","Price","Purchase Count"]]
profItem = items.groupby(["Item ID","Item Name"]).sum()
#
#profItem["Price"].astype(int)
profItem["Total Purchase Value"] = profItem["Price"]
profItem["Item Price"] = round(profItem["Price"]/profItem["Purchase Count"],2)
profItem["Item Price"] = profItem["Item Price"].map("${:.2f}".format)
#profItem.head()
profItem_ = profItem[["Purchase Count", "Item Price", "Total Purchase Value"]]
profItem__ = profItem_.sort_values("Total Purchase Value", ascending = False)
profItem__["Total Purchase Value"] = profItem__["Total Purchase Value"].map("${:.2f}".format)
profItem__.head()
                                                                           

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
